In [1]:
import os
import imageio
import mobie
import mobie.htm as htm
import mobie.metadata as metadata

import ome_zarr

In [2]:
from ome_zarr.io import parse_url
import zarr

In [3]:
from os.path import join

In [4]:
from mobie.metadata.project_metadata import create_project_metadata, add_dataset
from mobie.metadata.dataset_metadata import create_dataset_structure, create_dataset_metadata
from mobie.metadata.source_metadata import add_source_to_dataset, add_regions_to_dataset

In [5]:
from mobie.view_utils import create_view
from mobie.metadata import get_default_view, add_view_to_dataset

In [6]:
plate_path = "./zarr-files/Single-Plane.zarr/"
store = parse_url(plate_path, mode="w").store
    
plate = zarr.group(store=store)

In [7]:
mobie_project_folder = "./mobie-projects/2D-Demo"

In [8]:
dataset_name = "Single-Plane"

In [9]:
create_project_metadata(mobie_project_folder)

In [10]:
create_dataset_structure(mobie_project_folder, dataset_name, file_formats=["ome.zarr"])

'./mobie-projects/2D-Demo/Single-Plane'

In [11]:
create_dataset_metadata(join(mobie_project_folder, dataset_name), is2d=True)

In [12]:
add_dataset(mobie_project_folder, dataset_name, is_default=True)

In [13]:
from mobie.view_utils import create_grid_view

In [14]:
sources = {}
positions = {}
colors = {}
well_sources = {}
histograms = {}

In [15]:
import string
def to_position(well_name):
    r,c = well_name[0], well_name[1:]
    r = string.ascii_uppercase.index(r)
    c = int(c) - 1
    return [c, r]

In [16]:
def hex_to_rgba(h):
    return f"r={int(h[0:2], 16)},g={int(h[2:4], 16)},b={int(h[4:6], 16)},a=255"

In [17]:
from mobie.view_utils import create_view

In [18]:
from faim_hcs.UIntHistogram import UIntHistogram

In [19]:
for row in plate.group_keys():
    for col in plate[row].group_keys():
        attrs = plate[row][col][0].attrs.asdict()
        print(row + col.zfill(2))
        for i, ch in enumerate(attrs["omero"]["channels"]):
            print(ch["label"])
            if ch["label"] != "empty":
                label = ch["label"].replace(" ", "_")
                label = f"C{str(i).zfill(2)}_" + label
                name = label + "_" + row + col.zfill(2)
                path = join(plate.store.path, row, col, "0")
                hist = UIntHistogram.load(join(path, f"{label}_histogram.npz"))
                histograms[name] = hist
                add_source_to_dataset(join(mobie_project_folder, dataset_name), "image", name, file_format="ome.zarr", 
                         image_metadata_path=path, channel=i,
                                     view={})
                create_view(join(mobie_project_folder, dataset_name), view_name=name, 
                                                      sources=[[name]],
                    display_settings=[{"color": hex_to_rgba(ch["color"]), 
                                       "contrastLimits": [hist.quantile(0.01), hist.quantile(0.99)]}], 
                            display_group_names=[name],
                is_exclusive=False, menu_name="wells")

                well = row + col.zfill(2)
                if well not in well_sources.keys():
                    well_sources[well] = [name]
                else:
                    well_sources[well].append(name)

                if label not in sources.keys():
                    sources[label] = [name]
                else:
                    sources[label].append(name)

                if label not in positions.keys():
                    positions[label] = [to_position(row + col)]
                else:
                    positions[label].append(to_position(row + col))

                colors[label] = hex_to_rgba(ch["color"])
        print()

E07
FITC_05
FITC_05
FITC_05
empty

E08
FITC_05
FITC_05
FITC_05
empty



In [20]:
from mobie.view_utils import create_view

In [21]:
sources

{'C00_FITC_05': ['C00_FITC_05_E07', 'C00_FITC_05_E08'],
 'C01_FITC_05': ['C01_FITC_05_E07', 'C01_FITC_05_E08'],
 'C02_FITC_05': ['C02_FITC_05_E07', 'C02_FITC_05_E08']}

In [22]:
positions

{'C00_FITC_05': [[6, 4], [7, 4]],
 'C01_FITC_05': [[6, 4], [7, 4]],
 'C02_FITC_05': [[6, 4], [7, 4]]}

In [23]:
from mobie.metadata import get_grid_view, get_merged_grid_source_transform, get_image_display

In [24]:
trafos = [get_merged_grid_source_transform(sources=sources[k], 
                                           merged_source_name=f"plate_{k}", 
                                           positions=positions[k]) for k in sources.keys()]

In [25]:
trafos

[{'mergedGrid': {'sources': ['C00_FITC_05_E07', 'C00_FITC_05_E08'],
   'mergedGridSourceName': 'plate_C00_FITC_05',
   'positions': [[6, 4], [7, 4]]}},
 {'mergedGrid': {'sources': ['C01_FITC_05_E07', 'C01_FITC_05_E08'],
   'mergedGridSourceName': 'plate_C01_FITC_05',
   'positions': [[6, 4], [7, 4]]}},
 {'mergedGrid': {'sources': ['C02_FITC_05_E07', 'C02_FITC_05_E08'],
   'mergedGridSourceName': 'plate_C02_FITC_05',
   'positions': [[6, 4], [7, 4]]}}]

In [26]:
sources.keys()

dict_keys(['C00_FITC_05', 'C01_FITC_05', 'C02_FITC_05'])

In [27]:
histograms.keys()

dict_keys(['C00_FITC_05_E07', 'C01_FITC_05_E07', 'C02_FITC_05_E07', 'C00_FITC_05_E08', 'C01_FITC_05_E08', 'C02_FITC_05_E08'])

In [28]:
from copy import copy
def accumulate_histograms(histograms, prefix="DAPI"):
    histogram = None
    for k in histograms.keys():
        if k.startswith(prefix):
            if histogram is None:
                histogram = copy(histograms[k])
            else:
                histogram.combine(histograms[k])
        
    return histogram

In [29]:
displays = []
for k in sources.keys():
    plate_hist = accumulate_histograms(histograms, prefix=k)
    displays.append(get_image_display(k, 
                              [f"plate_{k}"], 
                              contrastLimits=[plate_hist.quantile(0.01), plate_hist.quantile(0.99)], 
                              color=colors[k], visible=True))

In [30]:
import pandas as pd

In [31]:
well_table = pd.DataFrame({"region_id": list(well_sources.keys()),
                          "treatment": ["Unknown",]*len(well_sources.keys())})

In [32]:
well_sources

{'E07': ['C00_FITC_05_E07', 'C01_FITC_05_E07', 'C02_FITC_05_E07'],
 'E08': ['C00_FITC_05_E08', 'C01_FITC_05_E08', 'C02_FITC_05_E08']}

In [33]:
add_regions_to_dataset(join(mobie_project_folder, dataset_name), "wells", well_table)

In [34]:
from mobie.metadata import get_region_display

In [35]:
displays.append(get_region_display("wells", well_sources, table_source="wells", lut="glasbey", opacity=0.5, visible=True))

In [36]:
displays

[{'imageDisplay': {'color': 'r=115,g=255,b=0,a=255',
   'contrastLimits': [142, 23030],
   'name': 'C00_FITC_05',
   'opacity': 1.0,
   'sources': ['plate_C00_FITC_05'],
   'visible': True}},
 {'imageDisplay': {'color': 'r=115,g=255,b=0,a=255',
   'contrastLimits': [115, 19267],
   'name': 'C01_FITC_05',
   'opacity': 1.0,
   'sources': ['plate_C01_FITC_05'],
   'visible': True}},
 {'imageDisplay': {'color': 'r=115,g=255,b=0,a=255',
   'contrastLimits': [142, 23021],
   'name': 'C02_FITC_05',
   'opacity': 1.0,
   'sources': ['plate_C02_FITC_05'],
   'visible': True}},
 {'regionDisplay': {'opacity': 0.5,
   'lut': 'glasbey',
   'name': 'wells',
   'sources': {'E07': ['C00_FITC_05_E07',
     'C01_FITC_05_E07',
     'C02_FITC_05_E07'],
    'E08': ['C00_FITC_05_E08', 'C01_FITC_05_E08', 'C02_FITC_05_E08']},
   'tableSource': 'wells',
   'visible': True}}]

In [37]:
default_view = {
    "isExclusive": True,
    "sourceDisplays": displays,
    "sourceTransforms": trafos,
    "uiSelectionGroup": "bookmark"
}

In [38]:
add_view_to_dataset(join(mobie_project_folder, dataset_name), "default", default_view)

In [39]:
mobie.validation.validate_project(mobie_project_folder)

Check views for dataset Single-Plane: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 68.58it/s]
Check view files for dataset Single-Plane: 0it [00:00, ?it/s]

The project at ./mobie-projects/2D-Demo is a valid MoBIE project.
